In [ ]:
# =========================================================
# Oil & Gas AI Analytics Platform
# Author: Hassan Gamal Albery
# =========================================================

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from statsmodels.tsa.arima.model import ARIMA
from pptx import Presentation
import io

# =========================================================
# PAGE CONFIG
# =========================================================
st.set_page_config(
    page_title="Oil & Gas AI Analytics",
    page_icon="🛢️",
    layout="wide"
)

# =========================================================
# ROLE BASED ACCESS (Streamlit-safe)
# =========================================================
ROLES = {
    "Viewer": ["Dashboard"],
    "Engineer": ["Dashboard", "ML & Forecast"],
    "Admin": ["Dashboard", "ML & Forecast", "Admin"]
}

def login():
    st.sidebar.title("🔐 Login")
    username = st.sidebar.text_input("Username")
    role = st.sidebar.selectbox("Role", list(ROLES.keys()))
    return username, role

# =========================================================
# LOAD EXCEL (ALL SHEETS)
# =========================================================
@st.cache_data
def load_excel(file):
    xls = pd.ExcelFile(file)
    return {s: pd.read_excel(xls, s) for s in xls.sheet_names}

# =========================================================
# WELL HEALTH SCORE (0–100)
# =========================================================
def health_score(net_bo, net_diff, wc):
    score = 100

    if net_bo <= 0:
        score -= 40
    elif net_bo < 100:
        score -= 20

    if net_diff < 0:
        score -= min(abs(net_diff) / 10, 30)

    if wc is not None:
        if wc > 80:
            score -= 30
        elif wc > 60:
            score -= 15

    return max(0, round(score))

# =========================================================
# ARIMA FORECASTING
# =========================================================
def forecast_series(series):
    try:
        model = ARIMA(series, order=(1, 1, 1))
        fit = model.fit()
        return fit.forecast(steps=5)
    except:
        return None

# =========================================================
# ML CLUSTERING (WELL TYPES)
# =========================================================
def cluster_wells(df):
    features = df[['Net BO', 'Net Diff BO']].fillna(0)
    kmeans = KMeans(n_clusters=3, random_state=42)
    df['Cluster'] = kmeans.fit_predict(features)
    return df

# =========================================================
# AUTO POWERPOINT GENERATOR
# =========================================================
def create_ppt(stats, df):
    prs = Presentation()

    slide = prs.slides.add_slide(prs.slide_layouts[0])
    slide.shapes.title.text = "Production Analytics Report"
    slide.placeholders[1].text = "Auto-generated by AI Analytics Platform"

    slide = prs.slides.add_slide(prs.slide_layouts[1])
    slide.shapes.title.text = "Executive Summary"
    tf = slide.placeholders[1].text_frame
    tf.text = (
        f"Total Wells: {stats['wells']}\n"
        f"Average Health Score: {stats['health']}\n"
        f"Zero Production Wells: {stats['zero']}"
    )

    slide = prs.slides.add_slide(prs.slide_layouts[1])
    slide.shapes.title.text = "High Risk Wells"
    risky = df[df['Health Score'] < 50]['Well'].astype(str).head(10)
    slide.placeholders[1].text_frame.text = ", ".join(risky)

    buffer = io.BytesIO()
    prs.save(buffer)
    buffer.seek(0)
    return buffer

# =========================================================
# MAIN APPLICATION
# =========================================================
def main():
    user, role = login()

    st.title("🛢️ Oil & Gas AI Analytics Platform")
    st.caption("Multi-sheet analytics • Forecasting • ML • Executive Reporting")

    uploaded_file = st.file_uploader(
        "📤 Upload Production Excel File",
        type=["xlsx", "xlsm"]
    )

    if not uploaded_file:
        st.info("Please upload a production Excel file to start.")
        return

    sheets = load_excel(uploaded_file)

    st.sidebar.subheader("📄 Sheets Detected")
    for s in sheets:
        st.sidebar.write(f"• {s}")

    # Use first sheet as main production sheet
    df = sheets[list(sheets.keys())[0]]

    # ============================
    # COLUMN MAPPING
    # ============================
    with st.sidebar:
        st.subheader("🧩 Column Mapping")
        field_col = st.selectbox("Field", df.columns)
        well_col = st.selectbox("Well", df.columns)
        net_bo_col = st.selectbox("Net BO", df.columns)
        net_diff_col = st.selectbox("Net Diff BO", df.columns)
        wc_col = st.selectbox("W/C (optional)", ["None"] + list(df.columns))
        wc_col = None if wc_col == "None" else wc_col

    df = df.rename(columns={
        field_col: "Field",
        well_col: "Well",
        net_bo_col: "Net BO",
        net_diff_col: "Net Diff BO"
    })

    if wc_col:
        df = df.rename(columns={wc_col: "WC"})

    df['Health Score'] = df.apply(
        lambda r: health_score(
            r['Net BO'],
            r['Net Diff BO'],
            r['WC'] if 'WC' in df else None
        ),
        axis=1
    )

    # ============================
    # DASHBOARD
    # ============================
    if "Dashboard" in ROLES[role]:
        st.header("📊 Production Dashboard")

        c1, c2, c3 = st.columns(3)
        c1.metric("Total Wells", len(df))
        c2.metric("Average Health Score", round(df['Health Score'].mean(), 1))
        c3.metric("Zero Production Wells", (df['Net BO'] == 0).sum())

        st.dataframe(df.sort_values("Health Score"))

        # Pareto Chart
        st.subheader("📈 Pareto Analysis (80/20)")
        d = df.sort_values("Net BO", ascending=False)
        d['CumPct'] = d['Net BO'].cumsum() / d['Net BO'].sum()
        fig, ax = plt.subplots()
        ax.plot(d['CumPct'], marker='o')
        ax.axhline(0.8, color='red', linestyle='--')
        ax.set_ylabel("Cumulative Production %")
        st.pyplot(fig)

    # ============================
    # ML & FORECAST
    # ============================
    if "ML & Forecast" in ROLES[role]:
        st.header("🧠 Machine Learning & Forecasting")

        df = cluster_wells(df)
        st.subheader("Well Clustering")
        st.dataframe(df[['Well', 'Cluster', 'Health Score']])

        st.subheader("🔮 Production Forecast")
        selected_well = st.selectbox("Select Well", df['Well'].unique())
        series = df[df['Well'] == selected_well]['Net BO']

        forecast = forecast_series(series)
        if forecast is not None:
            st.line_chart(pd.concat([series, forecast]))

    # ============================
    # ADMIN – POWERPOINT
    # ============================
    if "Admin" in ROLES[role]:
        st.header("📊 Auto PowerPoint Report")

        stats = {
            "wells": len(df),
            "health": round(df['Health Score'].mean(), 1),
            "zero": (df['Net BO'] == 0).sum()
        }

        ppt = create_ppt(stats, df)

        st.download_button(
            "📥 Download PowerPoint",
            ppt,
            "production_report.pptx",
            mime="application/vnd.openxmlformats-officedocument.presentationml.presentation"
        )

    st.success("✅ Analysis Completed Successfully")

# =========================================================
if __name__ == "__main__":
    main()
